In [1]:
import utils
import pandas as pd
import calendar

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# open files
arr = utils.fileList("rsc/")
arr.sort()
lists = list()
for item in arr:
    csv = utils.readCsv("rsc/" + item)
    csv.drop('rw', axis=1, inplace=True)
    csv.drop("jumlah_terdampak_rw", axis=1, inplace=True)
    csv.drop("jumlah_terdampak_rt", axis=1, inplace=True)
    csv.drop_duplicates(subset=None, keep='first', inplace=False)
    lists.append(csv)

In [5]:
nama_benahan = ["PULOGADUNG", "CILIINCING", "MAMPANG"]
for bulan in lists:
    for i in range(len(nama_benahan)):
        if i == 0:
            bulan[bulan['kecamatan'] == nama_benahan[i]] = "PULO GADUNG"
        elif i == 1:
            bulan[bulan['kecamatan'] == nama_benahan[i]] = "CILINCING"
        elif i == 2:
            bulan[bulan['kecamatan'] == nama_benahan[i]] = "MAMPANG PRAPATAN"

In [6]:
# normalize table in tanggal_kejadian column
new_list = list()
for bulan in lists:
    bulan['tanggal_kejadian'] = bulan['tanggal_kejadian'].str.split(',')
    bulan = bulan.explode('tanggal_kejadian').reset_index(drop=True)
    bulan['tanggal_kejadian'] = bulan['tanggal_kejadian'].str.extract('(\d+)', expand=False)
    new_list.append(bulan)


In [8]:
# making a dictionary of kelurahan, where kecamatan : [kelurahan]
kecamatan = list()
kelurahan = dict()
for bulan in new_list:
    kecamatan_bulan_n = bulan['kecamatan'].drop_duplicates()
    for i in kecamatan_bulan_n:
        if i not in kecamatan:
            kecamatan.append(i)
    num = int()
    for enum, kec in enumerate(kecamatan_bulan_n):
        kelurahan[kec] = list(bulan.loc[bulan['kecamatan'] == kec, 'kelurahan'].drop_duplicates())

In [9]:
print(kecamatan)

['JOHAR BARU', 'KEMAYORAN', 'SAWAH BESAR', 'TANAH ABANG', 'CILINCING', 'KELAPA GADING', 'KOJA', 'PENJARINGAN', 'PADEMANGAN', 'CENGKARENG', 'GROGOL PETAMBURAN', 'KALIDERES', 'KEBON JERUK', 'KEMBANGAN', 'PALMERAH', 'CILANDAK', 'JAGAKARSA', 'KEBAYORAN BARU', 'KEBAYORAN LAMA', 'MAMPANG PRAPATAN', 'PANCORAN', 'PASAR MINGGU', 'PESANGGRAHAN', 'SETIABUDI', 'TEBET', 'CAKUNG', 'CIPAYUNG', 'CIRACAS', 'DUREN SAWIT', 'JATINEGARA', 'KRAMAT JATI', 'MAKASAR', 'MATRAMAN', 'PASAR REBO', 'PULO GADUNG', 'SENEN', 'MENTENG', 'CEMPAKA PUTIH', 'GAMBIR', 'TANJUNG PRIOK', 'TAMAN SARI', 'TAMBORA', 'KEPULAUAN SERIBU SELATAN', 'KEPULAUAN SERIBU UTARA']


In [10]:
total = int()
for val in kelurahan.values():
    total += len(val)
print(total)

80


In [ ]:
for enum, kel in enumerate(kelurahan, 1):
    print("bulan " + str(calendar.month_name[enum]))
    for kec in kecamatan:
        print("kecamatan : " + kec)
        print("kelurahan : " + ", ".join(kelurahan[kec]))
        print("_________")
        

In [14]:
# kasus banjir per-kelurahan
kasus_banjir_per_bulan = list()
banjir_per_kelurahan = dict()

for bulan in new_list: 
    for kec in kecamatan:
        for kel in kelurahan[kec]:
            kejadian = len(bulan[bulan['kelurahan'] == kel])
            # print(kel + " " + str(kejadian))
            banjir_per_kelurahan[kel] = kejadian
    banjir_per_kelurahan = pd.Series(banjir_per_kelurahan).reset_index()
    banjir_per_kelurahan.columns = ['kelurahan', 'kejadian']
    kasus_banjir_per_bulan.append(banjir_per_kelurahan)
    banjir_per_kelurahan = dict()

In [15]:
for setiap_bulan in kasus_banjir_per_bulan:
    print(setiap_bulan.nlargest(5, "kejadian"))

          kelurahan  kejadian
20            PLUIT         5
61  CIPINANG MELAYU         4
30      LEBAK BULUS         3
40         RAWAJATI         3
57   KAMPUNG MELAYU         3
      kelurahan  kejadian
22        KAPUK        10
25   TEGAL ALUR         9
58  BIDARA CINA         8
60       CAWANG         8
67  PULO GADUNG         7
         kelurahan  kejadian
57  KAMPUNG MELAYU        10
60          CAWANG         9
58     BIDARA CINA         8
41   PEJATEN TIMUR         6
34   PONDOK PINANG         5
          kelurahan  kejadian
34    PONDOK PINANG         3
41    PEJATEN TIMUR         3
45       KEBON BARU         3
57   KAMPUNG MELAYU         3
61  CIPINANG MELAYU         3
         kelurahan  kejadian
41   PEJATEN TIMUR         4
40        RAWAJATI         3
45      KEBON BARU         3
57  KAMPUNG MELAYU         3
34   PONDOK PINANG         2
      kelurahan  kejadian
20        PLUIT         7
13      MARUNDA         3
22        KAPUK         3
76   PULAU PARI         3
18  KA